In [1]:
from keras import applications, optimizers
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '../dataset/train'
test_data_dir = '../dataset/test'
validation_data_dir = '../dataset/validation'
nb_train_samples = 1792
nb_test_samples = 210
nb_validation_samples = 525

batch_size = 16
epochs = 20

In [3]:
def buildModel():
    # build the VGG16 network
    base_model = applications.VGG16(weights='imagenet', 
                                    include_top=False,
                                    input_tensor=Input(shape=(img_width, img_height, 3)))

    for layer in base_model.layers:
        layer.trainable = False
        
    top_model = base_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(256, activation='relu')(top_model)
    top_model = Dense(6, activation='softmax')(top_model)
    
    model = Model(inputs=base_model.input, outputs=top_model)

    model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [4]:
def generateData(batch_size):
    train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        rescale=1. /255,
        rotation_range=30,
        shear_range=0.2,
        zoom_range=0.2)

    validation_datagen = ImageDataGenerator(rescale=1. /255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    
    return [train_generator,validation_generator]

In [5]:
def fineTuneModel(model, train_generator, validation_generator, epochs, batch_size):
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
    
    return model

In [6]:
def getMetrics(model, batch_size):
    
    test_datagen = ImageDataGenerator(rescale=1. /255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=1,
        class_mode='categorical')
    
    predicted_results = model.predict_generator(test_generator, 
                                                steps = nb_test_samples)
    predicted_results = np.argmax(predicted_results, axis=1)
    targets = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
    
    # confusion matrix
    print("CONFUSION MATRIX:")
    print(confusion_matrix(test_generator.classes, predicted_results))
    
    # classification report
    print("CLASSIFICATION REPORT:")
    print(classification_report(test_generator.classes, predicted_results, target_names=targets))

In [ ]:
model = buildModel()
train_generator,test_generator = generateData(batch_size)
trained_model = fineTuneModel(model,train_generator, test_generator, epochs, batch_size)
metrics = getMetrics(trained_model, batch_size)